In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from proj1_helpers import *
from implementations import *
DATA_FOLDER = '../data/'

In [3]:
from split_data import split_data

In [4]:
y_train, x_train, id_train = load_csv_data(DATA_FOLDER + 'train.csv')
y_test, x_test, id_test = load_csv_data(DATA_FOLDER + 'test.csv')

In [5]:
#x_train.filter(lambda v: v==v, x)

In [6]:
y_train

array([ 1., -1., -1., ...,  1., -1., -1.])

In [7]:
np.shape(x_train)

(250000, 30)

In [8]:
id_train

array([100000, 100001, 100002, ..., 349997, 349998, 349999])

### Basic Least Squares

In [5]:
def compare_prediction(w_train, x, y):
    y_pred = predict_labels(w_train, x)
    return (y_pred == y).sum()/len(y_pred)

In [11]:
w_ls, loss_ls = least_squares(y_train, x_train)
np.shape(w_ls)

(30,)

In [12]:
compare_prediction(w_ls, x_train, y_train)

0.74432799999999999

### Stochastic Gradient Descent

In [13]:
np.shape(np.asarray([1]*30)[:,None].reshape(30, 1))

(30, 1)

In [14]:
np.shape(np.asarray([1]*30))

(30,)

In [83]:
initial_w = np.asarray([1]*30)
max_iters = 100
gamma = 1.0e-6

In [84]:
w_sgd, loss_sgd = least_squares_SGD(y_train, x_train, initial_w, max_iters, gamma)


In [85]:
compare_prediction(w_sgd, x_train, y_train)

0.63684799999999997

### Ridge Regression

In [6]:
def compute_loss(y, tx, w):
    e = y - tx @ w
    return np.mean(e**2)/2.

In [7]:
def ridge_regression(y, tx, lambda_):

    first_term = tx.T@tx
    #sum ici les x*w
    left = first_term + lambda_ *np.identity(tx.shape[1])
    right = tx.T @ y
    w = np.linalg.solve(left, right)
    loss = compute_loss(y, tx, w)
    return w, loss;

In [8]:
from build_polynomial import build_poly

def ridge_regression_demo(x, y, degree, ratio, seed):
    """ridge regression demo."""
    # define parameter
    lambdas = np.logspace(-5, 0, 15)
    # split data
    x_tr, y_tr, x_te, y_te = split_data(x, y, ratio, seed)
    
    tr_poly = build_poly(x_tr, degree)
    te_poly = build_poly(x_te, degree)
    
    loss = 1000
    w = 0
    
    for lambda_ in lambdas:
        # ridge regression
        weight, l = ridge_regression(y_tr, tr_poly, lambda_)
        #weight = stack_w(weight, degree)
        mse_test = compute_loss(y_te, te_poly, weight)
        
        if(mse_test < loss):
            loss = mse_test
            w = weight
        
    return w, loss

In [94]:
seed = 88
degree = 6
split_ratio = 0.6


w_r, loss_ridge = ridge_regression_demo(x_train, y_train, degree, split_ratio, seed)

compare_prediction(w_r, build_poly(x_train, degree), y_train)

fin


0.79597200000000001

In [187]:
compare_prediction(w_r, build_poly(x_train, 6), y_train)

0.80079999999999996

### Rendu Ridge

In [188]:
y_pred = predict_labels(w_r, build_poly(x_test, 6))


In [191]:
create_csv_submission(id_test, y_pred, "ridge.csv")

### Ridge pour seeds et split_ratio

In [157]:
#seed = 95
seeds = np.arange(88,89,1)
degree = 6
split_ratio = np.arange(0.5,1,0.01)
best_compare_pred = 0

best_seed = 0
best_split = 0

for seed_ in seeds:
    print(seed_)
    for split_ in split_ratio:
        w_r, loss_ridge = ridge_regression_demo(x_train, y_train, degree, split_, seed_)
        
        if(type(w_r) is not int):
            comp_pred = compare_prediction(w_r.reshape(len(w_r), 1), build_poly(x_train, degree), y_train.reshape(len(y_train), 1))
    
            print(comp_pred)
            if(comp_pred > best_compare_pred):
                best_compare_pred = comp_pred
                best_seed = seed_
                best_split = split_
    

88
<class 'numpy.ndarray'>
0.7979


KeyboardInterrupt: 

In [151]:
#88
best_c

0.80079999999999996

### Cross-validation

In [28]:
from build_k_indices import build_k_indices
from implementations import ridge_regression
from build_polynomial import build_poly

def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression."""

    # get k'th subgroup in test, others in train
    train_indices = [x for j, x in enumerate(k_indices) if j != k]
    train_indices = [idx for part in train_indices for idx in part]
    test_indices = k_indices[k]
    
    x_tr = x[train_indices]
    y_tr = y[train_indices]
    
    x_te = x[test_indices]
    y_te = y[test_indices]

    # form data with polynomial degree
    #train_poly = build_poly(x_tr, degree)
    #test_poly = build_poly(x_te, degree)

    # ridge regression
    
    weights_train, loss_tr = ridge_regression(y_tr, x_tr, lambda_)

    # calculate the loss for train and test data
    loss_te = compute_loss(y_te, x_te, weights_train)

    return loss_tr, loss_te

In [29]:
def cross_validation_demo():
    seed = 55
    degree = 2
    k_fold = 4
    lambdas = np.logspace(-4, 0, 30)
    
    # split data in k fold
    k_indices = build_k_indices(y_train, k_fold, seed)
    
    min_mean_loss = 1000
    best_lambdas = np.zeros(k_fold)
    # cross validation
    for i_test in range(k_fold):
        min_loss = 1000
        for lambda_ in lambdas:
            loss_tr, loss_te = cross_validation(y_train, x_train, k_indices, i_test, lambda_, degree)
            
            if(loss_te < min_loss):
                min_loss = loss_te
                best_lambdas[i_test] = lambda_
    lambda_mean = np.mean(best_lambdas)
    return lambda_mean, min_mean_loss

In [30]:
bl, mml = cross_validation_demo()

In [31]:
bl

0.25007499999999999

In [32]:
w, l = ridge_regression(y_train, x_train, bl)

In [33]:
compare_prediction(w, x_train, y_train)

0.74440799999999996

### Methode for Logistic

shuffle_index separate the index on y if it's a 1 or -1 and then shuffle the indexes. At the end it choose a number of elems (same in both list) and take them. With this we will have the same mount of 1 and -1 and we will have better prediction

In [8]:
def shuffle_index_resize(y,x, max_elem_percent):
    y_pos = []
    y_neg = []
    
    max_elem = int(((max_elem_percent * len(y) ) / 100) / 2.0)

    for idx_y, val in enumerate(y):

        if(val == 1):
            y_pos.append(idx_y)
        else:
            y_neg.append(idx_y)
        
    data_size_pos = len(y_pos)  
    shuffle_indices_pos = np.random.permutation(np.arange(data_size_pos)) 
    data_size_neg = len(y_neg)  
    shuffle_indices_neg = np.random.permutation(np.arange(data_size_neg)) 
    new_y = []
    new_x = []
    for i in range(max_elem):
        new_y.append(y[y_pos[shuffle_indices_pos[i]]])
        new_y.append(y[y_neg[shuffle_indices_neg[i]]])
        new_x.append(x[y_pos[shuffle_indices_pos[i]]])
        new_x.append(x[y_neg[shuffle_indices_neg[i]]])
    
    return np.array(new_y), np.array(new_x)


def shuffle_index_resize2(y,x, max_elem_percent):
    y_pos = []
    y_neg = []
    
    max_elem = int(max_elem_percent / 2)

    for idx_y, val in enumerate(y):

        if(val == 1):
            y_pos.append(idx_y)
        else:
            y_neg.append(idx_y)

        
    data_size_pos = len(y_pos)  
    shuffle_indices_pos = np.random.permutation(np.arange(data_size_pos)) 
    data_size_neg = len(y_neg)  
    shuffle_indices_neg = np.random.permutation(np.arange(data_size_neg)) 
    new_y = []
    new_x = []
    for i in range(max_elem):
        new_y.append(y[y_pos[shuffle_indices_pos[i]]])
        new_y.append(y[y_neg[shuffle_indices_neg[i]]])
        new_x.append(x[y_pos[shuffle_indices_pos[i]]])
        new_x.append(x[y_neg[shuffle_indices_neg[i]]])
    
    return np.array(new_y), np.array(new_x)


new_y, new_x = shuffle_index_resize(y_train, x_train, 0.2)

np.shape(new_x)

(500, 30)

It was to predic y when use sigma between 0 and 1 to determine w

In [10]:
def compare_prediction2(w_train, x, y):
    y_pred = np.dot(x,w)
    y_pred[np.where(y_pred <= 0.5)] = 0
    y_pred[np.where(y_pred > 0.5)] = 1
    
    return (y_pred == y).sum()/len(y_pred)

### Logistics regression cross 


In [ ]:
def logistic_regression_gradient2(y, x, w, max_iter, degree, ratio, seed):
    """ridge regression demo."""
    # define parameter
    #gammas = np.logspace(-12, -9, 10)
    gammas = np.arange(1e-12,1e-11,1e-13)
    lambdas = [0.9]
    #lambdas = np.logspace(-7, 0, 30)
    # split data
    
    x_tr, y_tr, x_te, y_te = split_data(x, y, ratio, seed)
    
    best_comp = 0   
    weight = 0
    best_loss = 0
    w_init = w
    best_gamma = 0
    best_lambda = 0
    
    for gamma_ in gammas: 
        print("gamma")
        print(gamma_)
        for lambda_ in lambdas:
            w = w_init
            print("lambda")
            print(lambda_)
            for iter_ in range(max_iter):
                
                if((iter_ % 1000) == 0):
                    print(iter_)
                #loss, w = learning_by_gradient_descent(y, x, w, gamma_)
                loss, w = learning_by_penalized_gradient(y_tr, x_tr, w, gamma_, lambda_)
            
            print("ok")
            comp_pred = compare_prediction2(w, x_tr, y_tr)
            print(comp_pred)
            if(comp_pred > best_comp):
                best_comp = comp_pred
                print("best_comp")
                print(best_comp)
                weight = w
                #best_loss = loss
                best_gamma = gamma_
                best_lambda = lambda_
        
    return weight, best_loss

In [ ]:
def compare_prediction2(w_train, x, y):
    y_pred = np.dot(x,w)
    y_pred[np.where(y_pred <= 0.5)] = 0
    y_pred[np.where(y_pred > 0.5)] = 1
    
    return (y_pred == y).sum()/len(y_pred)

In [ ]:
seed = 88
degree = 6
split_ratio = 0.6
max_iter = 7000

y_train3 = y_train.reshape(len(y_train), 1)
tx = np.c_[np.ones((y_train3.shape[0], 1)), x_train]
w = np.zeros((tx.shape[1], 1))

for i in range(len(y_train3)):
    if(y_train3[i] < 0):
        y_train3[i] = 0


w_l, loss_log = logistic_regression_gradient2(y_train3, tx, w, max_iter, degree, split_ratio, seed)


compare_prediction2(w_l, tx, y_train3)

In [ ]:
compare_prediction2(w_l, tx, y_train3)

In [33]:
seed = 88
degree = 5
split_ratio = 0.6
max_iter = 10000

y_train3 = y_train.reshape(len(y_train), 1)
tx = np.c_[np.ones((y_train3.shape[0], 1)), x_train]
w = np.zeros((tx.shape[1], 1))

for i in range(len(y_train3)):
    if(y_train3[i] < 0):
        y_train3[i] = 0


w_l, loss_log = logistic_regression_gradient2(y_train3, tx, w, max_iter, degree, split_ratio, seed)


compare_prediction2(w_l, tx, y_train3)

gamma
1e-12
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
ok
0.68966
best_comp
0.68966
gamma
2.15443469003e-12
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
ok
0.692066666667
best_comp
0.692066666667
gamma
4.64158883361e-12
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
ok
0.696913333333
best_comp
0.696913333333
gamma
1e-11
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
ok
0.706013333333
best_comp
0.706013333333
gamma
2.15443469003e-11
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
ok
0.724386666667
best_comp
0.724386666667
gamma
4.64158883361e-11
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
ok
0.5455
gamma
1e-10
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
ok
0.704206666667
gamma
2.15443469003e-10
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
ok
0.513006666667
gamma
4.64158883361e-10
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
ok
0.691306666667
gamma
1e-09
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
ok
0.704166666667


0.70423999999999998

In [67]:
compare_prediction(w_r, tx, y_train3)

0.70957999999999999

### Cross Valid Logistic


In [6]:
def cross_valid_vizu(values, name, rmse_tr, rmse_te):
    """visualization the curves of mse_tr and mse_te."""
    plt.semilogx(values, rmse_tr, marker=".", color='b', label='train error')
    plt.semilogx(values, rmse_te, marker=".", color='r', label='test error')
    plt.xlabel(name)
    plt.ylabel("rmse")
    plt.title("cross validation")
    plt.legend(loc=2)
    plt.grid(True)
    plt.savefig("cross_validation")

In [36]:
from build_polynomial import *
def cross_validation_logi(y, x, k_indices, k, lambda_, degree, max_iters, gamma):
    """return the loss of ridge regression."""

    # get k'th subgroup in test, others in train
    train_indices = [x for j, x in enumerate(k_indices) if j != k]
    train_indices = [idx for part in train_indices for idx in part]
    test_indices = k_indices[k]
    
    x_tr = x[train_indices]
    y_tr = y[train_indices]
    
    x_te = x[test_indices]
    y_te = y[test_indices]
    
    y_tr = y_tr.reshape(len(y_tr), 1)
    #x_tr = np.c_[np.ones((y_tr.shape[0], 1)), x_tr]
    #initial_w = np.zeros((x_tr.shape[1], 1))
    
    # form data with polynomial degree
    #train_poly = build_poly(x_tr, degree)
    #test_poly = build_poly(x_te, degree)
    
    #initial_w = np.transpose(build_poly(np.transpose(initial_w), degree))
    
    x_te = np.c_[np.ones((y_te.shape[0], 1)), x_te]
    x_tr = np.c_[np.ones((y_tr.shape[0], 1)), x_tr]
    initial_w = np.zeros((x_tr.shape[1], 1))

    # ridge regression
    weights_train, loss_tr = reg_logistic_regression(y_tr, x_tr, lambda_, initial_w, max_iters, gamma)
    
    # calculate the loss for train and test data
    loss_te = calculate_loss(y_te, x_te, weights_train)

    return loss_tr, loss_te, weights_train

In [37]:
from build_k_indices import *

def cross_validation_logi_demo(lambdas, degrees, gammas, k_fold, max_iters):
    
    seed = 1
    # split data in k fold
    k_indices = build_k_indices(y_train[:4000], k_fold, seed)
    
    best_l = 0 
    best_deg = 0
    best_g = 0
    min_loss = 10000
    best_pred = 0
     # define list to store the variable
    rmse_tr = np.empty((len(lambdas), len(degrees), len(gammas)))
    rmse_te = np.empty((len(lambdas), len(degrees), len(gammas)))
    
    for ind_l, l in enumerate(lambdas):
        print("lambda_")
        print(ind_l)
        for ind_deg, deg in enumerate(degrees):
            for ind_g, g in enumerate(gammas):
                print("gamma")
                print(g)
                losses_tr_tmp = []
                losses_te_tmp = []
                for k_test in range(k_fold):
                    loss_tr, loss_te, we = cross_validation_logi(y_train[:100000],  \
                                                x_train[:100000], k_indices, k_test, l, deg, max_iters, g)
                    losses_tr_tmp.append(loss_tr)
                    losses_te_tmp.append(loss_te)
                mean_tr = np.mean(losses_tr_tmp)
                mean_te = np.mean(losses_te_tmp)
                rmse_t = np.sqrt(2*mean_te)
                rmse_tr[ind_l][ind_deg] = np.sqrt(2*mean_tr)
                rmse_te[ind_l][ind_deg] = rmse_t
                
                
                y_train2 = y_train.reshape(len(y_train), 1)
                x_train2 = np.c_[np.ones((y_train2.shape[0], 1)), x_train]
                
                pred = compare_prediction(we, x_train2, y_train2)
                
                print(pred)
                if(pred > best_pred):
                    best_pred = pred
                #print(rmse_t)
                #if(rmse_t < min_loss):
                    min_loss = rmse_t
                    best_l = l
                    best_deg = deg
                    best_g = g
     
    #cross_valid_vizu(lambdas, 'gamma', rmse_tr[0, 0, :], rmse_te[0, 0, :])
    #cross_valid_vizu(lambdas, 'gamma', rmse_tr[len(lambdas)-1, 0, :], rmse_te[len(lambdas)-1, 0, :])
    return best_l, best_deg, best_g

In [38]:
degrees = range(2, 3)
k_fold = 4
lambdas = [0.9]
gammas_ridge = np.logspace(-15, -8, 50)
max_iters = 1000

In [39]:
bl, bd, bg = cross_validation_logi_demo(lambdas, degrees, gammas_ridge, k_fold, max_iters)

lambda_
0
gamma
1e-15
0.63892
gamma
1.38949549437e-15
0.638928
gamma
1.93069772888e-15
0.638924
gamma
2.68269579528e-15
0.638936
gamma
3.72759372031e-15
0.638948
gamma
5.17947467923e-15
0.638952
gamma
7.19685673001e-15
0.638988
gamma
1e-14
0.63904
gamma
1.38949549437e-14
0.63908
gamma
1.93069772888e-14
0.63914
gamma
2.68269579528e-14
0.63922
gamma
3.72759372031e-14
0.639312
gamma
5.17947467923e-14
0.639424
gamma
7.19685673001e-14
0.639572
gamma
1e-13
0.639664
gamma
1.38949549437e-13
0.639896
gamma
1.93069772888e-13
0.640096
gamma
2.68269579528e-13
0.640276
gamma
3.72759372031e-13
0.640304
gamma
5.17947467923e-13
0.640424
gamma
7.19685673001e-13
0.640456
gamma
1e-12
0.640732
gamma
1.38949549437e-12
0.64142
gamma
1.93069772888e-12
0.643156
gamma
2.68269579528e-12
0.646584
gamma
3.72759372031e-12
0.652112
gamma
5.17947467923e-12
0.660636
gamma
7.19685673001e-12
0.66906
gamma
1e-11
0.676692
gamma
1.38949549437e-11
0.682388
gamma
1.93069772888e-11


/Users/kevinkappel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in sqrt


0.686872
gamma
2.68269579528e-11
0.68916
gamma
3.72759372031e-11
0.691572
gamma
5.17947467923e-11
0.693436
gamma
7.19685673001e-11
0.694984
gamma
1e-10
0.69644
gamma
1.38949549437e-10
0.697364
gamma
1.93069772888e-10
0.697932
gamma
2.68269579528e-10
0.698404
gamma
3.72759372031e-10
0.59024
gamma
5.17947467923e-10


/Users/kevinkappel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in sqrt


0.699064
gamma
7.19685673001e-10
0.529244
gamma
1e-09
0.694776
gamma
1.38949549437e-09
0.504304
gamma
1.93069772888e-09
0.498848
gamma
2.68269579528e-09
0.497084
gamma
3.72759372031e-09
0.69746
gamma
5.17947467923e-09
0.687728
gamma
7.19685673001e-09
0.494796
gamma
1e-08
0.698648


In [40]:
print(bl)
print(bd)
print(bg)

0
0
0


In [41]:
degree = 1
max_iter = 1000
gamma = 4.2e-10
lambda_ = 0.9

y_train2 = y_train.reshape(len(y_train), 1)
train_poly = build_poly(x_train, degree)

#train_poly = np.c_[np.ones((y_train2.shape[0], 1)), train_poly]
initial_w = np.zeros((train_poly.shape[1], 1))

w_l, loss_log = reg_logistic_regression(y_train2, train_poly, lambda_, initial_w, max_iters, gamma)

compare_prediction(w_l, train_poly, y_train2)

0.697932

In [153]:
degree = 1
max_iter = 1000
gamma = 4.2e-8
lambda_ = 0.9

print(np.shape(train_poly))
print(np.shape(w_l))
print(np.shape(y_train2))
compare_prediction(w_l, train_poly, y_train2)

(250000, 61)
(61, 1)
(250000, 1)


0.14075599999999999